Importing required packages

In [1]:
from newsapi import NewsApiClient
import csv
import pandas as pd
from textblob import TextBlob
from dateutil import parser
api=NewsApiClient(api_key='5474adf03fac4b10ad2b144b7fdf8732')

Pulling data

In [2]:
#pulling all articles related to marijuana from Canadian sources
all_articles = api.get_everything(q='marijuana',
                                      sources='the-globe-and-mail,financial-post',
                                      language='en')

In [3]:
all_articles

{'status': 'ok',
 'totalResults': 15,
 'articles': [{'source': {'id': 'the-globe-and-mail',
    'name': 'The Globe And Mail'},
   'author': 'Ian Irvine',
   'title': 'Doug Ford almost got it right on cannabis - The Globe and Mail',
   'description': "The Globe and Mail Doug Ford almost got it right on cannabis The Globe and Mail Ontario Premier Doug Ford should be congratulated for improving Ontario's marijuana market structure last week. He decided to privatize the retailing of marijuana in the province.…",
   'url': 'https://www.theglobeandmail.com/business/commentary/article-doug-ford-almost-got-it-right-on-cannabis/',
   'urlToImage': 'https://www.theglobeandmail.com/resizer/Epw5wh6pznjIfauywOk_SADkjB4=/1200x0/filters:quality(80)/arc-anglerfish-tgam-prod-tgam.s3.amazonaws.com/public/4DTFQLBHMRHU5FMMHCL67ISRHE.jpg',
   'publishedAt': '2018-08-22T21:34:11Z',
   'content': 'Having gone some way to free up entrepreneurial energies and create an expansive marketplace, Doug Ford balked a

Formatting data

In [4]:
#saving articles to news_data dataframe
news_data=pd.DataFrame(all_articles['articles'])

In [5]:
#parsing out date and formatting into news_data dataframe
news_data['date']=news_data.apply(lambda x: parser.parse(x['publishedAt']).strftime('%Y-%m-%d'), axis=1)

In [6]:
#calculating polarity and subjectivity of articles using TextBlob Natural Language Processing (NLP)
news_data['polarity']=news_data.apply(lambda x: TextBlob(x['description']).sentiment.polarity, axis=1)
news_data['subjectivity']=news_data.apply(lambda x: TextBlob(x['description']).sentiment.subjectivity, axis=1)

In [7]:
news_data.head()

,author,content,description,publishedAt,source,title,url,urlToImage,date,polarity,subjectivity
0,Ian Irvine,Having gone some way to free up entrepreneuria...,The Globe and Mail Doug Ford almost got it rig...,2018-08-22T21:34:11Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Doug Ford almost got it right on cannabis - Th...,https://www.theglobeandmail.com/business/comme...,https://www.theglobeandmail.com/resizer/Epw5wh...,2018-08-22,0.142857,0.301190
1,None,Part of Cannabis and your health With recreati...,The Globe and Mail Can cannabis contribute to ...,2018-08-21T09:04:00Z,"{'id': 'the-globe-and-mail', 'name': 'The Glob...",Can cannabis contribute to improving women's s...,https://www.theglobeandmail.com/cannabis/artic...,https://www.theglobeandmail.com/resizer/W1Zpp6...,2018-08-21,0.183333,0.800000
2,The Canadian Press,OTTAWA — The Second Cup Ltd. says it is active...,The coffee shop company says it has more than ...,2018-08-15T23:06:20Z,"{'id': 'financial-post', 'name': 'Financial Po...",Second Cup looking to convert Ontario coffee s...,https://business.financialpost.com/cannabis/se...,https://financialpostcom.files.wordpress.com/2...,2018-08-15,0.500000,0.500000
3,Bloomberg News,U.S. investors are piling into an exchange-tra...,The US$436 million ETFMG Alternative Harvest E...,2018-08-29T19:08:00Z,"{'id': 'financial-post', 'name': 'Financial Po...",American investors pile into Canadian marijuan...,https://business.financialpost.com/investing/f...,https://financialpostcom.files.wordpress.com/2...,2018-08-29,0.000000,0.000000
4,Bloomberg News,The manager of the first Canadian mutual fund ...,The manager of the first Canadian mutual fund ...,2018-09-13T16:19:27Z,"{'id': 'financial-post', 'name': 'Financial Po...",Brace for ‘scary down days’ in cannabis stocks...,https://business.financialpost.com/investing/f...,https://financialpostcom.files.wordpress.com/2...,2018-09-13,0.250000,0.333333


In [8]:
#grouping by mean values of dates
news_scores= news_data.groupby(by= 'date').mean()
news_scores

,polarity,subjectivity
date,,
2018-08-15,0.500000,0.500000
2018-08-17,0.250000,0.333333
2018-08-21,0.091667,0.400000
2018-08-22,0.142857,0.301190
2018-08-28,0.000000,0.066667
2018-08-29,0.000000,0.000000
2018-08-31,0.208333,0.725000
2018-09-04,0.166667,0.166667
2018-09-06,0.000000,0.000000


In [9]:
#filtering for dates
news_scores_filtered=news_scores.loc['2018-08-01':'2018-09-01']
#writing to csv file
news_scores_filtered.to_csv('newsscores.csv',sep=',')